<a href="https://colab.research.google.com/github/Y-YHat/dexter_ai/blob/main/agent_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
requirements.txt
* langchain
* langchain-experimental
* youtube-search
* duckduckgo-search
* wikipedia
* langchain-community
* langchain-openai

```







In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [19]:
!pip install -q -r '/content/drive/My Drive/requirements.txt'

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


```
imports
```

In [43]:
import os
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor
from langchain.agents.react.agent import create_react_agent
import pandas as pd
import numpy as np
import random
from math import comb
from IPython.display import Markdown
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from datetime import datetime, timedelta

In [21]:

from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [22]:
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [25]:
# set up the agent
llm = ChatOpenAI(model_name="gpt-4", max_tokens=500, temperature=0.7)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [27]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "What is the weather like in London today? Get the current weather for today's date",
)



> Entering new AgentExecutor chain...
Thought: The assistant doesn't have real-time data capabilities, so it can't get the current weather in London. However, it can use the tavily_search_results_json tool to fetch the most recent information available on the internet. 

Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "current weather in London"
  }
}
```
Observation: [{'url': 'https://world-weather.info/forecast/united_kingdom/london/june-2024/', 'content': 'Extended weather forecast in London. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ London Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://world-weather.info/forecast/united_kingdom/london/30-june/', 'content': 'Detailed ⚡ Weather Forecast for June 30 in London, England, United Kingdom - 🌡️ temperature, wind, atmospheric pressure, humidity and precipitations - World-Weather.info. Add the current city. Search. Weath

'The current weather in London is 15.3°C (59.5°F), with partly cloudy conditions. The wind is coming from the NNW at a speed of 9.0 kph (5.6 mph), and the atmospheric pressure is 1016.0 mb.'

In [29]:
# Wiki tool for info retrieval
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

# tool to search youtube videos
tool_youtube = YouTubeSearchTool()

# web search tool
search = DuckDuckGoSearchRun()

# toolset
tools = [tool_wiki, tool_youtube, search]

In [30]:
from langchain.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt= PromptTemplate.from_template(template)

In [32]:
agent = create_react_agent(llm, tools, prompt)
brixo  = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
brixo.invoke({"input":
    """What would be a nice movie to watch on a warm summer evening. Follow these steps.

    First, decide which movie you would recommend.

    Second, show me the trailler video of the movie that you suggest.

    Finally, tell me why you chose the movie.
    """})



> Entering new AgentExecutor chain...
I need to suggest a movie that would be nice to watch on a warm summer evening. For a summer movie, a light-hearted comedy or a romantic movie would be a good choice. Let's suggest the movie "500 Days of Summer" which is a romantic comedy and is also fitting for the summer theme.
Action: youtube_search
Action Input: 500 Days of Summer,1['https://www.youtube.com/watch?v=PsD0NpFSADM&pp=ygUSNTAwIERheXMgb2YgU3VtbWVy']This is the trailer for the movie "500 Days of Summer". Now I need to explain why I chose this movie.
Final Answer: I recommend the movie "500 Days of Summer". You can watch the trailer here: https://www.youtube.com/watch?v=PsD0NpFSADM&pp=ygUSNTAwIERheXMgb2YgU3VtbWVy. I chose this movie because it's a romantic comedy that's both light-hearted and introspective, perfect for a warm summer evening. The movie uniquely explores a romantic relationship, making it interesting and relatable. Plus, the title itself suggests it's a great summer mo

{'input': 'What would be a nice movie to watch on a warm summer evening. Follow these steps.\n\n    First, decide which movie you would recommend.\n\n    Second, show me the trailler video of the movie that you suggest.\n\n    Finally, tell me why you chose the movie.\n    ',
 'output': 'I recommend the movie "500 Days of Summer". You can watch the trailer here: https://www.youtube.com/watch?v=PsD0NpFSADM&pp=ygUSNTAwIERheXMgb2YgU3VtbWVy. I chose this movie because it\'s a romantic comedy that\'s both light-hearted and introspective, perfect for a warm summer evening. The movie uniquely explores a romantic relationship, making it interesting and relatable. Plus, the title itself suggests it\'s a great summer movie!'}

In [39]:
# Function to generate random timestamps
def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

# Parameters
num_records = 100
num_beacon_records = 50
start_date = datetime.now() - timedelta(days=1)
end_date = datetime.now()
beacon_interval = timedelta(minutes=5)  # Beacons every 5 minutes

# Generate synthetic firewall data
firewall_data = {
    "timestamp": [random_date(start_date, end_date) for _ in range(num_records)],
    "source_ip": [f"192.168.1.{random.randint(1, 255)}" for _ in range(num_records)],
    "dest_ip": [f"10.0.0.{random.randint(1, 255)}" for _ in range(num_records)],
    "bytes": [random.randint(40, 1500) for _ in range(num_records)],
    "packets": [random.randint(1, 10) for _ in range(num_records)],
    "domain": [f"domain{random.randint(1, 100)}.com" for _ in range(num_records)],
    "label": 0  # Label normal traffic as 0
}

firewall_df = pd.DataFrame(firewall_data)

# Generate synthetic beacon data (beacons to newly registered domains)
beacon_data = {
    "timestamp": [],
    "source_ip": [],
    "dest_ip": [],
    "bytes": [],
    "packets": [],
    "domain": [],
    "label": []  # Label beacon traffic as 1
}

start_beacon_time = start_date
for _ in range(num_beacon_records):
    beacon_data["timestamp"].append(start_beacon_time)
    beacon_data["source_ip"].append(f"192.168.1.{random.randint(1, 255)}")
    beacon_data["dest_ip"].append(f"10.0.0.{random.randint(1, 255)}")
    beacon_data["bytes"].append(random.randint(40, 1500))
    beacon_data["packets"].append(random.randint(1, 10))
    beacon_data["domain"].append(f"newdomain{random.randint(1, 100)}.com")
    beacon_data["label"].append(1)  # Label beacon traffic as 1
    start_beacon_time += beacon_interval  # Increment time by beacon interval

beacon_df = pd.DataFrame(beacon_data)

# Combine the dataframes
combined_df = pd.concat([firewall_df, beacon_df], ignore_index=True)

# Display the first few rows of the combined DataFrame
print('shape:', combined_df.shape)
combined_df.head()

shape: (150, 7)


,timestamp,source_ip,dest_ip,bytes,packets,domain,label
0,2024-06-30 10:33:35.100034,192.168.1.173,10.0.0.134,591,4,domain10.com,0
1,2024-06-30 11:23:00.100034,192.168.1.22,10.0.0.13,1047,10,domain60.com,0
2,2024-06-30 06:35:07.100034,192.168.1.39,10.0.0.227,1172,10,domain1.com,0
3,2024-06-30 01:20:29.100034,192.168.1.116,10.0.0.134,820,9,domain1.com,0
4,2024-06-29 22:23:39.100034,192.168.1.212,10.0.0.32,1426,9,domain99.com,0


In [41]:
prefix = """ Use the dataset provided. """

dataqio = create_pandas_dataframe_agent(
    llm,
    combined_df,
    verbose=True,
    max_iterations=3,
    prefix=prefix,
    allow_dangerous_code=True, # adding this to enable the feature to run - "opt in"
    agent_executor_kwargs={
        "handle_parsing_errors": True
    }
)

In [42]:
response = dataqio.invoke("What is the dataset about?")
display(Markdown(f"{response}"))



> Entering new AgentExecutor chain...
Thought: The dataset seems to be about network traffic with various details such as timestamp, source ip, destination ip, bytes, packets, domain and a label column.
Final Answer: The dataset is about network traffic logs.

> Finished chain.


{'input': 'What is the dataset about?', 'output': 'The dataset is about network traffic logs.'}

In [50]:
response = dataqio.invoke("create a graph to show beaconing activity in the data")
display(Markdown(f"{response}"))



> Entering new AgentExecutor chain...
Thought: To create a graph showing beaconing activity, I first need to understand what beaconing activity is. Beaconing is a pattern of network traffic where an internal host sends out packets at a consistent interval to an external entity. It's often associated with malware or botnets communicating with a command and control server. 

In the context of this dataset, we might identify beaconing by looking for repeated communication between the same source and destination IP addresses, with consistent packet sizes.

Action: python_repl_ast
Action Input: df.groupby(['source_ip', 'dest_ip', 'bytes']).size()source_ip      dest_ip     bytes
192.168.1.1    10.0.0.190  326      1
192.168.1.100  10.0.0.144  315      1
192.168.1.109  10.0.0.198  463      1
192.168.1.116  10.0.0.134  820      1
               10.0.0.167  980      1
                                   ..
192.168.1.91   10.0.0.239  932      1
192.168.1.92   10.0.0.71   926      1
192.168.1.96

{'input': 'create a graph to show beaconing activity in the data', 'output': 'Agent stopped due to iteration limit or time limit.'}